In [1]:
# mnist_cnn_keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
print(tf.keras.__version__)

2.3.0
2.4.0


In [2]:
# mnist 데이터 가져오기
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()

# X값의 타입을 float형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
x_test = tf.cast(x_test,dtype=tf.float32)

# X값의 shape을 4차원으로 변환
x_train = tf.reshape(x_train,[-1,28,28,1])
x_test = tf.reshape(x_test,[-1,28,28,1])

print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)
(60000,) (10000,)


In [3]:
# MNIST 분류 CNN keras 모델 구현
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28, 28, 1),kernel_size=(3,3),filters=32,padding='same'),
    tf.keras.layers.Dense(units=10,activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),

    tf.keras.layers.Conv2D(input_shape=(14, 14, 1),kernel_size=(3,3),filters=64,padding='same'),
    tf.keras.layers.Dense(units=10,activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),  
    
    tf.keras.layers.Conv2D(input_shape=(7, 7, 1),kernel_size=(3,3),filters=128,padding='same'),
    tf.keras.layers.Dense(units=10,activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'), 
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=10,activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.summary()

# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
# tf.keras.layers.Conv2D(
#     filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
#     dilation_rate=(1, 1), groups=1, activation=None, use_bias=True,
#     kernel_initializer='glorot_uniform', bias_initializer='zeros',
#     kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
#     kernel_constraint=None, bias_constraint=None, **kwargs
# )

# https://yeomko.tistory.com/40
# Xavier Glorot Initialization : W(Weight) 값을 fan_in,fan_out를 사용하여 초기화하여 정확도 향상

# loss 종류
# mean_squared_error : 평균제곱 오차
# binary_crossentropy : 이진분류 오차
# categorical_crossentropy : 다중 분류 오차. one-hot encoding 클래스, [0.2, 0.3, 0.5] 와 같은 출력값과 실측값의 오차값을 계산한다.
# sparse_categorical_crossentropy: 다중 분류 오차. 위와 동일하지만 , integer type 클래스라는 것이 다르다.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
dense (Dense)                (None, 28, 28, 10)        330       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        5824      
_________________________________________________________________
dense_1 (Dense)              (None, 14, 14, 10)        650       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 10)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         1

In [4]:
model.fit(x_train,y_train,epochs=25,validation_split=0.25)
# validation_split : 학습데이터를 학습과 평가용 데이터로 분리(75%:25%)

Epoch 1/25
1407/1407 [==============================] - 14s 10ms/step - loss: 0.3273 - accuracy: 0.8995 - val_loss: 0.1077 - val_accuracy: 0.9671
Epoch 2/25
1407/1407 [==============================] - 14s 10ms/step - loss: 0.1266 - accuracy: 0.9618 - val_loss: 0.1245 - val_accuracy: 0.9603
Epoch 3/25
1407/1407 [==============================] - 15s 10ms/step - loss: 0.1074 - accuracy: 0.9668 - val_loss: 0.0930 - val_accuracy: 0.9705
Epoch 4/25
1407/1407 [==============================] - 14s 10ms/step - loss: 0.0959 - accuracy: 0.9698 - val_loss: 0.0763 - val_accuracy: 0.9760
Epoch 5/25
1407/1407 [==============================] - 14s 10ms/step - loss: 0.0896 - accuracy: 0.9711 - val_loss: 0.0836 - val_accuracy: 0.9741
Epoch 6/25
1407/1407 [==============================] - 14s 10ms/step - loss: 0.0787 - accuracy: 0.9756 - val_loss: 0.0720 - val_accuracy: 0.9772
Epoch 7/25
1407/1407 [==============================] - 14s 10ms/step - loss: 0.0763 - accuracy: 0.9757 - val_loss: 0.0755 -

In [5]:
model.evaluate(x_test,y_test,verbose=0)
# [0.05205076187849045, 0.9836999773979187]

[0.05205076187849045, 0.9836999773979187]

In [6]:
# VGGNet (VGG-19) 스타일의 MNIST 분류 CNN 모델 
#--------------------------------------------
# ( Conv2D * 2개  --> MaxPool2D ) * 2회 : 4층
# ( Conv2D * 4개  --> MaxPool2D ) * 3회 : 12층
# Dense * 3개                           : 3층
#--------------------------------------------
#                                     총 19층
#--------------------------------------------
# 각 네트워크마다 필터의 수를 2배로 증가 시킨다 : 32-->64-->128-->256-->512

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 256)        

In [7]:
model.fit(x_train,y_train,epochs=25,validation_split=0.25)

Epoch 1/25
1407/1407 [==============================] - 23s 16ms/step - loss: 0.6575 - accuracy: 0.8464 - val_loss: 0.0743 - val_accuracy: 0.9771
Epoch 2/25
1407/1407 [==============================] - 23s 16ms/step - loss: 0.1804 - accuracy: 0.9491 - val_loss: 0.0665 - val_accuracy: 0.9801
Epoch 3/25
1407/1407 [==============================] - 23s 17ms/step - loss: 0.1401 - accuracy: 0.9616 - val_loss: 0.0473 - val_accuracy: 0.9863
Epoch 4/25
1407/1407 [==============================] - 23s 16ms/step - loss: 0.1190 - accuracy: 0.9677 - val_loss: 0.0512 - val_accuracy: 0.9856
Epoch 5/25
1407/1407 [==============================] - 23s 16ms/step - loss: 0.1119 - accuracy: 0.9695 - val_loss: 0.0445 - val_accuracy: 0.9877
Epoch 6/25
1407/1407 [==============================] - 23s 17ms/step - loss: 0.1093 - accuracy: 0.9705 - val_loss: 0.0446 - val_accuracy: 0.9885
Epoch 7/25
1407/1407 [==============================] - 23s 17ms/step - loss: 0.1083 - accuracy: 0.9712 - val_loss: 0.0449 -

In [8]:
model.evaluate(x_test,y_test,verbose=0)
# [0.031050674617290497, 0.9919999837875366]  : 정확도 향상,코랩에서 실행

[0.031050674617290497, 0.9919999837875366]